## **Calculating Ground Deformation Using Remote Sensing Data: A Python Project**
* Author: Shushant Koirala

**Abstract**

The Python script focuses on automating the process of analyzing ground deformation using LiDAR data. This script utilizes various libraries, including tkinter for creating a user-friendly interface, arcpy for processing LiDAR data and calculating Digital Terrain Models (DTMs), rasterio for handling raster data, numpy for numerical calculations, matplotlib for visualization, and earthpy for geospatial data processing. The script's key functionalities involve enabling users to select pre-event and post-event data folders through a graphical interface. It then processes LAS files, computes the difference between pre-event and post-event DTMs to determine deformation, reclassifies deformation data into distinct ranges, and finally visualizes the results using color maps and scale bars. Additionally, it generates a comprehensive summary text file documenting the entire process. Its implementation is organized into six primary steps: GUI creation, DTM calculation, difference computation, reclassification, visualization, and summary creation. These steps ensure a systematic workflow, simplifying the process for users and enhancing efficiency. The benefits of employing this script include automation of manual tasks, thereby saving time and reducing human error. There are few manual inputs required as of now, which will be removed with the further enhancement of the code.

**Introduction:**

In recent decades, land deformation has emerged as a critical threat, driven by diverse natural and human-induced environmental processes (Awasthi et al., 2022). This Python script is designed to calculate ground deformation displacement using remote sensing data. Ground deformation refers to the movement of the Earth's surface, which can be caused by a variety of factors, such as earthquakes, landslides, and mining activities. Remote sensing data, such as LiDAR, can be used to measure ground deformation by tracking changes in the Earth's surface over time (Okyay et al., 2019). The main objective of the project involves using remote sensing data to measure the change in position of the Earth's surface between two different points in time.The secondary objective focuses on creating a Python program that can be used to automate the process of calculating ground deformation displacement.The script is designed to allow users to easily adapt it to their specific needs.

Compared to "out-of-the-box" tools in ArcGIS/QGIS, this Python script offers several advantages:

1. The script can automate entire workflows, including data selection, processing, analysis, and visualization.
2. The script automates repetitive tasks and optimizes processing steps, leading to faster results.
3. The script allows for easy execution through a single interface, without requiring complex scripting knowledge.

**Study Area and Data**

The temporal LiDAR dataset with the **.las** extension has been downloaded from the website (https://portal.opentopography.org/dataCatalog). The python script is independent of the study area. New Zeland and Japan, are two different study area used as test study area.

**Sample Data**

The LiDAR data with **.las** extension for two study area, New Zeland and Japan, are used as the input data. 

**Project Design:**

The script is divided into six main steps:

1. Create Graphical User Interfaces (GUIs) to select input LiDAR data and output folder
2. Calculate Digital Terrain Model (DTM) from LiDAR Data
3. Calculate Differences between pre- and post-DTMs
4. Reclassify the difference raster
5. Visualize the reclassified raster
6. Create Summary Report


**Results and Discussion**

**Step 1: Create Graphical User Interfaces (GUIs)**
* Import `tkinter` module to select folders interactively.
* Define function to select the folder location of Pre-Event Data and Post-Event Data, and location to save the output.

In [ ]:
try:
    import tkinter as tk
    from tkinter import filedialog
    
except:
    !pip install tkinter
    
    import tkinter as tk
    from tkinter import filedialog

In [ ]:
def select_folder(message):
    root = tk.Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory(title=message)
    return folder_selected

In [ ]:
pre_event_files = select_folder("Select Pre-Event Data Folder")
post_event_files = select_folder("Select Post-Event Data Folder")
output_folder = select_folder("Select Output Folder to Save Processed Data")

**Step 2: Calculate Digital Terrain Model (DTM)**
* Import `arcpy` and `os` module
* Define class to Process LiDAR Data
* Calculate DTM

    *Note: It is necessary to add ArcGIS 'arcpy' path in the environment variable.*

In [ ]:
try:
    import arcpy
    import os
    from arcpy.sa import *
    
except:
    !pip install os
    
    import arcpy
    import os
    from arcpy.sa import *

In [ ]:
class LASProcessor:
    def __init__(self, input_folder, output_folder):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.output_files = []  # List to store processed files

    def process_las_files(self, output_prefix, is_pre_event):
        arcpy.env.overwriteOutput = True
        os.chdir(self.output_folder)

        for item in os.listdir(self.input_folder):
            if item.endswith(".las"):
                las_layer = os.path.join(self.output_folder, item[:-4] + " Layer")
                processed_file = self.process_event_data(os.path.join(self.input_folder, item), las_layer, output_prefix, is_pre_event)
                if processed_file:
                    self.output_files.append(processed_file)

        return self.output_files  # Return the list of processed files

    def process_event_data(self, input_path, output_layer, output_prefix, is_pre_event):
        arcpy.management.MakeLasDatasetLayer(input_path, output_layer, class_code=[2, 6])
        outRaster_DTM = os.path.join(self.output_folder, output_prefix + "_DTM" + ".tif")
        DSM = arcpy.conversion.LasDatasetToRaster(output_layer, outRaster_DTM, 'ELEVATION',
                                                  'BINNING AVERAGE NATURAL_NEIGHBOR', 'FLOAT', 'CELLSIZE', 1, 1)

        return outRaster_DTM

**Step 3: Calculate Difference**
* Define function to calculate the difference between pre-event and post-event DTMs

In [ ]:
def calculate_difference(pre_event_file, post_event_file, output_folder):
    # Load pre-event and post-event rasters
    pre_event_raster = Raster(pre_event_file)
    post_event_raster = Raster(post_event_file)

    # Calculate the difference (post_event_DTM - pre_event_DTM)
    difference_raster = post_event_raster - pre_event_raster

    # Save the difference raster
    output_difference = os.path.join(output_folder, "deformation.tif")
    difference_raster.save(output_difference)

    return output_difference

In [ ]:
# Instantiate the LASProcessor class for pre-event and process LAS files with output prefixes
pre_event_DTM = LASProcessor(pre_event_files, output_folder).process_las_files("pre_event", is_pre_event=True)

# Instantiate the LASProcessor class for post-event and process LAS files with output prefixes
post_event_DTM = LASProcessor(post_event_files, output_folder).process_las_files("post_event", is_pre_event=False)

# Calculate the difference between post-event DSM and pre-event DTM
if pre_event_DTM and post_event_DTM:
    output_difference = calculate_difference(pre_event_DTM[-1], post_event_DTM[-1], output_folder)
    print(f"Difference saved at: {output_difference}")

else:
    print("No processed files found.")

**Step 4: Calculate Maximum and Minimum Value**
* Install and Import `rasterio` and `numpy` module

In [ ]:
try:
    import rasterio as rio
    import numpy as np
        
except:
    !pip install rasterio
    !pip install numpy

    import rasterio as rio
    import numpy as np
    

In [ ]:
with rio.open(output_difference) as src:
    arr = src.read(1)
    
    min_value = np.nanmin(arr)
    max_value = np.nanmax(arr)

    print(min_value)
    print(max_value)

* Define Lower and Upper Bound value

In [ ]:
#Assign the range of lower and upper bound depending upon the study area requirement. 
range_lowerbound = -0.05
range_upperbound = 0.05

**Step4: Reclassify**
* Reclass the difference raster based on lower and upper range

In [ ]:
outReclass3 = Reclassify(output_difference, "Value", 
                         RemapRange([[min_value,range_lowerbound,1],[range_lowerbound,range_upperbound,2],[range_upperbound,max_value,3]]), "NODATA")
#Save the reclass file and also make sure to change the output folder location to your desired location.
reclass_file = r"D:\UNT\1. First Semester\GEOG 5560 - Python\Project\Study Area\Central Otago, Otago, New Zealand\3. Outputs\reclass.tif"
outReclass3.save(reclass_file)


**Step 5: Visualize reclassed raster**
* Install and Import `matplotlib` and `earthpy`

In [ ]:
try:
    import matplotlib.pyplot as plt
    import earthpy.plot as ep
    from matplotlib.colors import ListedColormap
    from matplotlib.patches import Patch
    from matplotlib_scalebar.scalebar import ScaleBar
        
except:
    !pip install matplotlib
    !pip install earthpy
    !pip install matplotlib_scalebar

    import matplotlib.pyplot as plt
    import earthpy.plot as ep
    from matplotlib.colors import ListedColormap
    from matplotlib.patches import Patch
    from matplotlib_scalebar.scalebar import ScaleBar



In [ ]:
with rio.open(reclass_file) as src:
    arr = src.read(1)
    bounds = src.bounds

    #Define Colors for different classes
    colors = ['#ffffff', '#F5F500', '#750288', '#F50000']

    cmap = ListedColormap(colors)

    f, ax = plt.subplots(figsize=(10, 12))
    im = ax.imshow(arr, cmap=cmap)
    
    #Define the labels for classified maps
    legend_labels = ["<-0.05", "-0.05 to 0.05", ">0.05"]
    legend_patches = [Patch(color=color, label=label) for color, label in zip(colors[1:], legend_labels)]

    #Define Legent Title
    legend_title = "Deformation (m)"
    legend = ax.legend(handles=legend_patches, bbox_to_anchor=(1, 1), loc='upper left', title=legend_title)

    #Add Northarrow
    x, y, arrow_length = 0.99, 0.99, 0.075
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
    arrowprops=dict(facecolor='black', width=1, headwidth=10),
    ha='center', va='center', fontsize=10,
    xycoords=ax.transAxes)

    #Add Scalebar
    plt.setp(legend.get_title(), fontsize='12')
    scalebar = ScaleBar(0.5, location='lower right')
    ax.add_artist(scalebar)

    plt.axis("off")
    plt.title("Ground Deformation using DTM",fontsize=18)
    plt.tight_layout()
    plt.show()


**Step 6: Create Summary**
* Import `datetime`
* Define function to create summary text file
* Include details about 'LAS Data Processing', 'Deformation Calculation', and 'Deformation Reclassification'

In [ ]:
try:
    from datetime import datetime

        
except:
    !pip install datetime
    
    from datetime import datetime

In [ ]:
def create_summary_text(output_difference, reclass_file, min_value, max_value, start_time):
    output_file = "summary.txt"
    with open(output_file, "w") as file:
        file.write("Summary of Processes\n")
        file.write("--------------------\n\n")

        file.write("1. LAS Data Processing:\n")
        file.write(f"   - Pre-Event Data Folder: {pre_event_files}\n")
        file.write(f"   - Post-Event Data Folder: {post_event_files}\n")
        file.write(f"   - Output Folder for Processed Data: {output_folder}\n\n")

        file.write("2. Deformation Calculation:\n")
        file.write(f"   - Difference Raster Saved at: {output_difference}\n\n")

        file.write("3. Deformation Reclassification:\n")
        file.write(f"   - Reclassified File Saved at: {reclass_file}\n")
        file.write(f"   - Minimum Deformation Value: {min_value}\n")
        file.write(f"   - Maximum Deformation Value: {max_value}\n\n")

        end_time = datetime.now()
        file.write("Summary Generated at: {}\n".format(end_time.strftime("%Y-%m-%d %H:%M:%S")))

# Get the start time
start_time = datetime.now()

create_summary_text(output_difference, reclass_file, min_value, max_value, start_time)

**Conclusion:**

The Python script for calculating ground deformation displacement is a valuable tool for researchers and professionals in the field of geospatial analysis. The developed program is designed to create an elevation model (DTM) autonomously. This Python-based program operates without reliance on specific study areas. The primary manual involvement in the project includes providing the reclass range and file name parameters for reclass.Further enhancements will be implemented to minimize manual input. A detailed analysis will be conducted to identify and comprehend patterns of change in Earth's surface. Additionally, efforts will be made to make the output more interactive, aiming to facilitate a more engaging and user-friendly experience.

**References:**

Awasthi, S., Varade, D., Bhattacharjee, S., Singh, H., Shahab, S., &amp; Jain, K. (2022). Assessment of land deformation and the associated causes along a rapidly developing Himalayan foothill region using multi-temporal sentinel-1 sar datasets. Land, 11(11), 2009. https://doi.org/10.3390/land11112009 

Okyay, U., Telling, J., Glennie, C. L., &amp; Dietrich, W. E. (2019). Airborne lidar change detection: An overview of Earth Sciences Applications. Earth-Science Reviews, 198, 102929. https://doi.org/10.1016/j.earscirev.2019.102929 